# Stormpy bindings

### Simulator

In [7]:
import stormpy
import stormpy.examples
import stormpy.examples.files
import stormpy.simulator

path = stormpy.examples.files.prism_dtmc_die
prism_program = stormpy.parse_prism_program(path)
model = stormpy.build_model(prism_program)

simulator = stormpy.simulator.create_simulator(model, seed=42)


In [8]:
simulator.restart()

(0, [0.0], {'init'})

In [9]:
simulator.step()

(2, [1.0], set())

In [10]:
simulator.step()

(5, [1.0], set())

In [11]:
simulator.step()

(11, [1.0], {'done', 'five'})

In [12]:
model.labeling.get_labels_of_state(11)

{'done', 'five'}

### Model checker and policy output

In [16]:

from utils import *

grid_size = 6
file_name = 'storm_files/delivery'
formula_str = "Rmin=? [F \"goal\"]"
program = stormpy.parse_prism_program(file_name+'.prism')
formulas = stormpy.parse_properties(formula_str,program)
options = stormpy.BuilderOptions([f.raw_formula for f in formulas])
print(dir(options))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'preserved_label_names', 'set_add_out_of_bounds_state', 'set_add_overlapping_guards_label', 'set_build_all_labels', 'set_build_all_reward_models', 'set_build_choice_labels', 'set_build_observation_valuations', 'set_build_state_valuations', 'set_build_with_choice_origins']


In [17]:
options.set_build_state_valuations()
options.set_build_choice_labels()
options.set_build_all_labels()
options.set_build_all_reward_models()

### Delivery model

In [18]:
with open('storm_files/delivery.prism') as f:
    print(f.read())

mdp

const int RADIUS=2;
const int N=6;
const int xMAX = N-1;
const int yMAX = N-1;
const int aXMAX = xMAX;
const int aYMAX = yMAX;
const int aXMIN = 0;
const int aYMIN = 1;
const int dxMAX = xMAX;
const int dyMAX = yMAX;
const int dxMIN = 0;
const int dyMIN = 0;
const double slippery=0;

formula done = start & pickup & dx = 0 & dy = 0;
formula collect = start & dx = dxMAX & dy = dyMAX;
formula crash = (dx=ax) & (ay=dx);
formula seedrone = ax-dx < RADIUS + 1 & dx-ax < RADIUS +1 & ay-dy < RADIUS+1 & dy -ay < RADIUS+1;



module master
    start : bool init false;
    turn : bool init false;

    [placement] !start -> (start'=true);
    [north] start & !done & turn -> (turn'=!turn);
    [south] start & !done & turn -> (turn'=!turn);
    [east]  start & !done & turn -> (turn'=!turn);
    [west]  start & !done & turn -> (turn'=!turn);
    [adv]  start & !done & !turn -> (turn'=!turn);
endmodule


module drone
    dx : [dxMIN..dxMAX] init 1;
    dy : [dyMIN..dyMAX] init 1;
    pickup : bool

### Build and check model

In [19]:
model = stormpy.build_sparse_model_with_options(program,options)
result = stormpy.model_checking(model,formulas[0],extract_scheduler=True)
drone_states = [s for s in  model.states if len(s.actions)>1]
print(result.at(0))

1.1339165719957613


### Pull out the policy

Choices for each state:
    
    def get_choice_at(sched,s):
        return sched.get_choice(s).get_deterministic_choice()

Choice labels:
    
    def get_choice_label_at(model,sched,s):
        if len(get_choice_labels(model,s))>1:
            return get_choice_labels(model,s)[get_choice_at(sched,s)]

In [20]:
drone_choices = {storm_in_to_idx(model,s,grid_size):get_choice_label_at(model,result.scheduler,s) for s in drone_states}
for d_c in drone_choices:
    print(f'{d_c}: {drone_choices[d_c]}')

(7, 13, 0): south
(7, 15, 0): east
(7, 8, 0): south
(7, 20, 0): south
(1, 12, 0): south
(1, 14, 0): south
(1, 7, 0): south
(1, 19, 0): south
(13, 12, 0): south
(13, 14, 0): east
(13, 7, 0): south
(13, 19, 0): south
(8, 12, 0): south
(8, 14, 0): south
(8, 7, 0): south
(8, 19, 0): south
(6, 12, 0): south
(6, 14, 0): east
(6, 7, 0): south
(6, 19, 0): south
(1, 16, 0): east
(1, 9, 0): east
(1, 21, 0): south
(13, 16, 0): east
(13, 9, 0): east
(13, 21, 0): east
(8, 16, 0): east
(8, 9, 0): south
(8, 21, 0): south
(6, 16, 0): east
(6, 9, 0): east
(6, 21, 0): east
(1, 8, 0): east
(13, 8, 0): south
(8, 8, 0): south
(6, 8, 0): east
(1, 26, 0): south
(13, 26, 0): south
(8, 26, 0): south
(6, 26, 0): south
(1, 13, 0): south
(1, 6, 0): south
(1, 18, 0): south
(7, 12, 0): south
(7, 6, 0): south
(7, 18, 0): south
(2, 12, 0): south
(2, 13, 0): south
(2, 6, 0): south
(2, 18, 0): south
(0, 12, 0): south
(0, 13, 0): south
(0, 6, 0): south
(0, 18, 0): south
(1, 15, 0): east
(1, 20, 0): south
(2, 15, 0): sou

### Simulator for MDP

In [35]:
import random
random.seed(23)
prism_program = stormpy.parse_prism_program('storm_files/slipgrid.nm')

formula_str = "Rmin=? [F \"goal\"]"
formulas = stormpy.parse_properties(formula_str,prism_program)

model = stormpy.build_model(prism_program)
simulator = stormpy.simulator.create_simulator(model, seed=42)

options = stormpy.BuilderOptions()
options.set_build_choice_labels()
options.set_build_state_valuations()
model = stormpy.build_sparse_model_with_options(prism_program, options)
simulator = stormpy.simulator.create_simulator(model, seed=42)
simulator.set_action_mode(stormpy.simulator.SimulatorActionMode.GLOBAL_NAMES)
simulator.set_observation_mode(stormpy.simulator.SimulatorObservationMode.PROGRAM_LEVEL)
# 3 paths of at most 20 steps.
paths = []
for m in range(3):
    path = []
    state, reward, labels = simulator.restart()
    path = [f"({state['x']},{state['y']})"]
    for n in range(20):
        actions = simulator.available_actions()
        select_action = random.randint(0,len(actions)-1)
        path.append(f"--{actions[select_action]}-->")
        state, reward, labels = simulator.step(actions[select_action])
        path.append(f"({state['x']},{state['y']})")
        if simulator.is_done():
            break
    paths.append(path)
for path in paths:
    print(" ".join(path))
    print()


(1,1) --west--> (1,2) --south--> (1,2) --south--> (2,2) --east--> (2,1) --south--> (3,1) --south--> (3,1) --west--> (3,2) --east--> (3,1) --north--> (2,1) --west--> (2,2) --south--> (2,2) --east--> (2,2) --west--> (2,3) --north--> (1,3) --south--> (2,3) --west--> (2,4) --north--> (2,4) --north--> (1,4) --south--> (1,4) --east--> (1,3)

(1,1) --west--> (1,1) --south--> (1,1) --west--> (1,2) --east--> (1,1) --south--> (1,1) --south--> (2,1) --north--> (1,1) --west--> (1,2) --west--> (1,3) --south--> (2,3) --south--> (3,3) --east--> (3,2) --east--> (3,1) --west--> (3,2) --east--> (3,1) --west--> (3,1) --south--> (3,1) --west--> (3,1) --north--> (2,1) --south--> (3,1)

(1,1) --south--> (2,1) --west--> (2,2) --west--> (2,2) --west--> (2,2) --south--> (2,2) --south--> (3,2) --south--> (4,2) --west--> (4,2) --east--> (4,1) --north--> (3,1) --west--> (3,2) --north--> (2,2) --north--> (1,2) --south--> (2,2) --west--> (2,3) --north--> (1,3) --west--> (1,4) --east--> (1,3) --west--> (1,4) --east-

### Simulate an implemented policy

In [53]:
result = stormpy.model_checking(model,formulas[0],extract_scheduler=True)
simulator.set_observation_mode(stormpy.simulator.SimulatorObservationMode.STATE_LEVEL)
simulator.set_action_mode(stormpy.simulator.SimulatorActionMode.INDEX_LEVEL)
# 3 paths of at most 20 steps.
paths = []
for m in range(3):
    path = []
    state, reward, labels = simulator.restart()
    state_des = model.state_valuations.get_json(state)
    path = [f"({state_des['x']},{state_des['y']})"]
    for n in range(20):
        actions = simulator.available_actions()
        choice = result.scheduler.get_choice(state)
        path.append(f"--{choice.get_deterministic_choice()}-->")
        state, reward, labels = simulator.step(choice.get_deterministic_choice())
        state_des = model.state_valuations.get_json(state)
        path.append(f"({state_des['x']},{state_des['y']})")
        if simulator.is_done():
            break
    paths.append(path)
for path in paths:
    print(" ".join(path))
    print()


(1,1) --0--> (2,1) --1--> (2,1) --1--> (3,1) --1--> (4,1) --1--> (4,2) --2--> (4,2) --2--> (4,3) --2--> (4,4) --0--> (4,4) --0--> (3,4) --1--> (3,4) --1--> (3,4) --1--> (3,4) --1--> (4,4) --0--> (3,4) --1--> (3,4) --1--> (4,4) --0--> (4,4) --0--> (3,4) --1--> (3,4)

(1,1) --0--> (2,1) --1--> (2,1) --1--> (2,1) --1--> (2,1) --1--> (2,1) --1--> (3,1) --1--> (3,1) --1--> (4,1) --1--> (4,2) --2--> (4,2) --2--> (4,3) --2--> (4,3) --2--> (4,4) --0--> (3,4) --1--> (3,4) --1--> (4,4) --0--> (3,4) --1--> (3,4) --1--> (3,4) --1--> (3,4)

(1,1) --0--> (2,1) --1--> (2,1) --1--> (3,1) --1--> (4,1) --1--> (4,2) --2--> (4,2) --2--> (4,2) --2--> (4,3) --2--> (4,4) --0--> (4,4) --0--> (3,4) --1--> (3,4) --1--> (3,4) --1--> (4,4) --0--> (3,4) --1--> (4,4) --0--> (3,4) --1--> (4,4) --0--> (3,4) --1--> (4,4)

